In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))  # adjust '..' as needed
if project_root not in sys.path:
    sys.path.append(project_root)

In [ ]:
import evaluate # Hugging Face splits metrics into a separate package
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import plotly.express as px
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import Dataset

In [ ]:
# -------------------------------
# Load CSVs
# -------------------------------
train_df = pd.read_csv("../data/train_data.csv").dropna()
val_df   = pd.read_csv("../data/val_data.csv").dropna()
test_df  = pd.read_csv("../data/test_data.csv").dropna()

# Get all unique labels from training set
label_names = sorted(train_df['label'].unique())  # e.g. ['FAKE','REAL','NEUTRAL','OTHER']
label2id = {name: i for i, name in enumerate(label_names)}
id2label = {i: name for name, i in label2id.items()}

print('label_names: ', label_names)
print('label2id: ', label2id)
print('id2label: ', id2label)

# Apply mapping to all splits
for df in [train_df, val_df, test_df]:
    df['label'] = df['label'].map(label2id)

X_train = train_df['text']
y_train = train_df['label'].values

X_val = val_df['text']
y_val = val_df['label'].values

X_test = test_df['text']
y_test = test_df['label'].values

# For Hugging Face dataset format
from datasets import Dataset

train_dataset = Dataset.from_dict({'text': X_train, 'label': y_train})
val_dataset   = Dataset.from_dict({'text': X_val, 'label': y_val})
test_dataset  = Dataset.from_dict({'text': X_test, 'label': y_test})

In [ ]:
# Get class weights -- there's an imbalance in the dataset
from sklearn.utils.class_weight import compute_class_weight
import torch

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

class_weights = torch.tensor(class_weights, dtype=torch.float)
print("Final class weights:", class_weights)

In [ ]:
# -------------------------------
# Fine-tuning pretrained model
# -------------------------------
import numpy as np
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from transformers import pipeline, Trainer

model_name = "distilbert-base-uncased"; model_name_out = "distilbert-base-uncased"
# model_name = "bert-base-uncased"; model_name_out = "bert-base-uncased"
# model_name = "google/electra-base-discriminator"; model_name_out = "electra-base-discriminator"
# model_name = "answerdotai/ModernBERT-base"; model_name_out = "Modernbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model     = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(np.unique(y_train)))

# Tokenization
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset   = val_dataset.map(tokenize, batched=True)
test_dataset  = test_dataset.map(tokenize, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    learning_rate=1e-5,
    load_best_model_at_end=True, # keep the best model
    metric_for_best_model="f1_weighted",
    greater_is_better=True,
    report_to="none", # disable wandb, tensorboard, etc.
    dataloader_pin_memory=False,
)

In [ ]:
from src.utils import compute_metrics
from src.models import WeightedTrainer
import torch.nn as nn

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#trainer = Trainer(
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    class_weights=class_weights,
)

trainer.train()

# save best model
model_path = "./results/transformer_model/" + model_name_out + "/"
os.makedirs(model_path, exist_ok=True)

trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
# Compute optimal threshold
from src.threshold import find_optimal_threshold, plot_threshold_optimization

false_label_id = label2id["LIKELY_FALSE"]

result = find_optimal_threshold(
    model=model,
    tokenizer=tokenizer,
    val_dataset=val_dataset,
    false_label_id=false_label_id
)

In [ ]:
best_threshold = result["best_threshold"]
plot_threshold_optimization(result, 'cost')

print("Optimal Threshold:", best_threshold)

# Save threshold and label maps
import json
with open(os.path.join(model_path, "threshold.json"), "w") as f:
    json.dump({"best_threshold": best_threshold}, f)

with open(os.path.join(model_path, "label_map.json"), "w") as f:
    json.dump({'label2id':label2id, 'id2label':id2label},f)

In [ ]:
# -------------------------------
# Evaluation on val set (apply thresholded predictions and compute metrics)
# -------------------------------
from scipy.special import softmax
from src.utils import compute_metrics
import json
import numpy as np

# Get raw predictions from the trainer
preds_output = trainer.predict(val_dataset)
logits = preds_output.predictions  # shape (N, num_labels)
probs = softmax(logits, axis=1)

# Probability of the 'likely_false' class (assumed to be class 0)
p_false = probs[:, 0]

# Use the previously computed best threshold
threshold = best_threshold
y_pred = np.where(p_false >= threshold, 0, 1)

# Map numeric preds to label names for downstream reporting
id2label_names = {v: k for k, v in label2id.items()}  # numeric to name
y_pred_labels = [id2label_names[int(i)] for i in y_pred]
y_val_labels = [id2label_names[int(i)] for i in y_val]

# Compute metrics using the updated compute_metrics (accepts 1D preds)
metrics_thresholded = compute_metrics((y_pred, np.array(y_val)))
print('Metrics (compute_metrics) for thresholded predictions:')
print(json.dumps(metrics_thresholded, indent=2))

In [ ]:
# -------------------------------
# Confusion Matrix
# -------------------------------
from src.viz import * 

label_order = ["LIKELY_FALSE", "LIKELY_TRUE"]

# Overall confusion matrix
plotly_confusion_matrix(y_val_labels, y_pred_labels, label_order, title="Overall Confusion Matrix")

cm = confusion_matrix(y_val_labels, y_pred_labels)
tn, fp, fn, tp = cm.ravel()
print('tn, fp, fn, tp:', tn, fp, fn, tp)

# cm_df = pd.DataFrame(cm, index=np.unique(y_val_labels), columns=np.unique(y_val_labels))
# fig = px.imshow(cm_df,
#                 text_auto=True,
#                 labels=dict(x="Predicted", y="Actual", color="Count"),
#                 color_continuous_scale="Blues",
#                 title="Confusion Matrix")
# fig.show()

In [ ]:
# -------------------------------
# Word Clouds for each label
# -------------------------------
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# First, create a DataFrame with text + predicted label
val_texts = val_df["text"].tolist()
pred_labels = y_pred_labels

df_pred = pd.DataFrame({
    "text": val_texts,
    "pred": pred_labels
})

label_names = np.unique(y_pred_labels)

# Generate a word cloud per label
for lbl in label_names:
    texts = " ".join(df_pred[df_pred["pred"]==lbl]["text"])

    # Create the word cloud
    wc = WordCloud(width=800, height=400, background_color="white", colormap="coolwarm").generate(texts)

    plt.figure(figsize=(12,6))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Word Cloud for label: {lbl}")
    plt.show()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

df_pred = pd.DataFrame({
    "text": val_texts,
    "pred": y_pred_labels
})

# Colors per label
label_colors = {
    "LIKELY_TRUE": "blue",
    "LIKELY_FALSE": "red"
    # "SUPPORTS": "seagreen",
    # "REFUTES": "crimson",
    # "NEUTRAL": "gray",
    # "DISPUTED": "orange"
}

for lbl in np.unique(y_pred_labels):
    texts = " ".join(df_pred[df_pred["pred"]==lbl]["text"])
    if not texts.strip():
        print(f"No predictions for label {lbl}, skipping word cloud.")
        continue

    word_freq = Counter(texts.split())

    wc = WordCloud(
        width=800,
        height=400,
        background_color="white",
        max_words=100,
        color_func=lambda *args, **kwargs: label_colors[lbl]  # pass label directly here
    ).generate_from_frequencies(word_freq)

    plt.figure(figsize=(12,6))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Word Cloud for label: {lbl}", fontsize=16)
    plt.show()